#### Урок 3. Парсинг HTML. BS, SQLAlchemy

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [146]:
# Установка БД
"""
docker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo
docker ps -a
docker start f48a0957b452
docker exec -it f48a0957b452 mongo

"""

'\ndocker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo\ndocker ps -a\ndocker start f48a0957b452\ndocker exec -it f48a0957b452 mongo\n\n'

In [15]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from pprint import pprint

client = MongoClient('mongodb://127.0.0.1:27017')
vacancy_pd = pd.read_pickle('vacancy_pd.pkl')

In [16]:
vacancy_pd
type(vacancy_pd["min_salary"][3])

list

In [17]:
def db_vacancy_writer(df):
    client.vacancy.docs.drop()
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    result = docs.insert_many(df.to_dict(orient='records'))
    return result.inserted_ids

In [18]:
client.vacancy.docs.drop()
a = db_vacancy_writer(vacancy_pd)


In [19]:
def db_vacancy_search(salary):
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    collections = docs.find({ "min_salary": {'$gte':salary} }).sort("min_salary")
    for x in collections:
        print(*x['vacancy'], ": зарплата от ", *x['min_salary'], " до ",*x['max_salary'])
        
    a = docs.find({ "max_salary": {'$gte':salary} } ).sort("max_salary")# 
    for x in a:
        print(*x['vacancy'], ": зарплата до ",*x['max_salary'])



In [20]:
salary = 100000
db_vacancy_search(salary)

Системный аналитик : зарплата от  100000  до  150000
Программист 1C : зарплата от  100000  до  200000
Старший маркетолог-аналитик : зарплата от  105000  до  nan
Ведущий специалист по внутреннему аудиту : зарплата от  110000  до  nan
Начальник отдела финансового мониторинга : зарплата от  110000  до  nan
Врач-методист : зарплата от  110000  до  nan
Менеджер проектного офиса : зарплата от  115000  до  nan
Методолог-консультант : зарплата от  120000  до  120000
Методолог : зарплата от  120000  до  nan
Руководитель проектного офиса : зарплата от  120000  до  160000
Системный аналитик : зарплата от  130000  до  160000
Системный аналитик (чат-боты) : зарплата от  149500  до  195000
Начальник отдела проектного офиса (IT-компания) : зарплата от  150000  до  200000
Бизнес-аналитик : зарплата от  150000  до  nan
Заместитель главного бухгалтера по работе с дочерними обществами : зарплата от  180000  до  nan
Методолог-консультант : зарплата до  100000
Менеджер проектов : зарплата до  100000
Внутре

In [21]:
# Пока не получилость сделать проверку если не указана минимальная ЗП, а максимальная больше-равно заданной

salary = 100000
vacancy_db = client['vacancy']
docs = vacancy_db.docs

a = docs.find({ "min_salary": {}})

#a = docs.find({ "max_salary": {'$gte':salary} and "min_salary": {'nan'}}).sort("max_salary") 
print(a)
for x in a:
#    print(x)
    print(*x['vacancy'], ": зарплата от ", *x['min_salary'], " до ",*x['max_salary'])




In [ ]:
#objects = docs.find()
#pprint(objects)
#for obj in objects:
    
#pprint(obj)

In [ ]:
### пример

import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df